In [1]:
include("./Bilevel/Bilevel.jl")
include("utils.jl")
using .Bilevel
using Plots
using ArgParse, PrettyTables

In [ ]:
# -- DEFAULT PARAMETERS OF A SIMULATION --
verbose     = true

# ------- Choice of the model -------
bilevel     = false 
storage     = false 

# ------- Profiles parameters -------
nb_days     = 2 
delta_t     = 60 
peak_power  = 7.0 
EV          = false 
EHP         = false 

# ------- Users parameters -------
PV_pen        = 1.0     
storage_pen   = 1.0   
storage_eff   = 0.98
storage_cost  = 500.0   # kEUR/MVA
amort_storage = 15      # years
PV_capa       = 0.4     # max in MVA
PV_cost       = 500.0   # EUR/kWc 
PV_conv_cost  = 200.0   # kEUR/MVA
amort_PV_conv = 10      # years
amort_PV      = 25      # years 
EIC           = 0.3     # EUR/kWh  
EEC           = 0.1     # EUR/kWh   
DSOEC         = 0.1     # EUR/kWh  
GCC           = 80.0    # EUR/kVA_year
cos_phi       = 0.95  

# ------- DSO parameters -------
substation_cost   = 1e3  # kEUR/MVA
amort_DSO         = 50  
interest_rate_DSO = 0.06 
money_basis = 1.0
weight_I = weight_V = 1.0e-2
weight_obj1 = weight_obj2 = 1.0

In [7]:
include("main_bilevel.jl")

#--------------------------------------------------------------------------------
#                                                                                
#                                  Bilevel DNEP                                   
#                                                                                
#                                Graduation work                                 
#                                                                                
#--------------------------------------------------------------------------------
# @ Manon Cornet

Running a simulation with the following characteristics:
--------------------------------------------------------

╭────────────┬───────┬───────┬─────────┬─────────┬─────────┬─────────┬─────────┬───────────╮
│ Model Type │    EV │   EHP │ Delta_t │ PV_CAPA │     EIC │     EEC │   DSOEC │       GCC │
│        [-] │   [-] │   [-] │   [min] │   [MVA] │ [€/kWh] │ [€/kWh] │ [€/kWh] │ [€/kVA/y] │
├────────────

Simulation(Network(Main.Bilevel.Line[Main.Bilevel.Line(Main.Bilevel.Edge(1, Main.Bilevel.Node(1, (x = 0.0, y = 0.1)), Main.Bilevel.Node(3, (x = 0.1, y = 0.1))), 0.1, false, nothing, nothing, nothing, nothing, nothing, nothing, nothing), Main.Bilevel.Line(Main.Bilevel.Edge(2, Main.Bilevel.Node(2, (x = 0.0, y = 0.0)), Main.Bilevel.Node(3, (x = 0.1, y = 0.1))), 0.14142135623730953, false, nothing, nothing, nothing, nothing, nothing, nothing, nothing), Main.Bilevel.Line(Main.Bilevel.Edge(3, Main.Bilevel.Node(3, (x = 0.1, y = 0.1)), Main.Bilevel.Node(4, (x = 0.2, y = 0.1))), 0.1, false, nothing, nothing, nothing, nothing, nothing, nothing, nothing)], Main.Bilevel.Bus[Main.Bilevel.Substation(Main.Bilevel.Node(1, (x = 0.0, y = 0.1)), false, (V_min = 0.95, V_max = 1.05), 15.0, 0.0, nothing, nothing, nothing), Main.Bilevel.Substation(Main.Bilevel.Node(2, (x = 0.0, y = 0.0)), false, (V_min = 0.95, V_max = 1.05), 15.0, 0.0, nothing, nothing, nothing), Main.Bilevel.User(Main.Bilevel.Node(3, (x = 0

In [8]:
model = Bilevel.build_model(simulation; set_names=true)
simulation.storage

Set parameter Username
Academic license - for non-commercial use only - expires 2023-11-10
Set parameter TimeLimit to value 600
Set parameter MIPGap to value 0.01
Set parameter MIPFocus to value 1


[ Info: Building model...
[ Info: Built model in 0.02 seconds


false

In [39]:
return_value = Bilevel.solve_model(model, formulation.powerflow)
save_struct(network_topology, "network_topology.json")
save_struct(network, "network_data.json")

Set parameter MIPGap to value 0.01
Set parameter MIPFocus to value 1
Set parameter TimeLimit to value 600
Set parameter MIPGap to value 0.01
Set parameter MIPFocus to value 1
Set parameter TimeLimit to value 600
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[arm])

CPU model: Apple M1 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 530 rows, 451 columns and 2068 nonzeros
Model fingerprint: 0x5f4dd400
Model has 20 quadratic constraints
Variable types: 434 continuous, 17 integer (17 binary)
Coefficient statistics:
  Matrix range     [1e-09, 2e+05]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [2e-02, 1e+00]
  Bounds range     [4e-01, 1e+00]
  RHS range        [8e-02, 1e+01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 246 rows and 183 columns
Presolve time: 0.01s
Presolved: 284 rows, 268 columns, 1265 nonzeros
Presolved model has 20 qua

In [40]:
thesis_dir = "/Users/manoncornet/Documents/University/TFE/ThesisWriting/Master_Thesis/figures" 
for t in 1:model[:time_steps]
    Bilevel.print_network_tikz(model[:network_data], t, 40, 40; dir=thesis_dir, filename="OneLevel_graph_$(t)")
end

TulipDaisyThis is LuaHBTeX, Version 1.15.0 (TeX Live 2022) 
 restricted system commands enabled.
(./OneLevel_graph_1.tex
LaTeX2e <2021-11-15> patch level 1
 L3 programming layer <2022-02-24>
(/usr/local/texlive/2022/texmf-dist/tex/latex/standalone/standalone.cls
Document Class: standalone 2018/03/26 v1.3a Class to compile TeX sub-files stan
dalone
(/usr/local/texlive/2022/texmf-dist/tex/latex/tools/shellesc.sty)
(/usr/local/texlive/2022/texmf-dist/tex/generic/iftex/ifluatex.sty
(/usr/local/texlive/2022/texmf-dist/tex/generic/iftex/iftex.sty))
(/usr/local/texlive/2022/texmf-dist/tex/latex/xkeyval/xkeyval.sty
(/usr/local/texlive/2022/texmf-dist/tex/generic/xkeyval/xkeyval.tex
(/usr/local/texlive/2022/texmf-dist/tex/generic/xkeyval/xkvutils.tex
(/usr/local/texlive/2022/texmf-dist/tex/generic/xkeyval/keyval.tex))))
(/usr/local/texlive/2022/texmf-dist/tex/latex/standalone/standalone.cfg)
(/usr/local/texlive/2022/texmf-dist/tex/latex/base/article.cls
Document Class: article 2021/10/04 v1.4n 

In [41]:
function sig_round(x)
    for (i, a) in enumerate(x)
        if isa(a, AbstractFloat)
            x[i] = round(a, sigdigits=4)
            if abs(a) < 1e-3
                x[i] = 0
            end
        end
    end
    return x
end

sig_round (generic function with 1 method)

In [29]:
network         = model[:network_data]
buses           = network.buses
N               = get_nb_buses(network)
Ns              = get_nb_substations(network)
T               = model[:time_steps]
P_cons = [buses[i].load_profile.time_serie[t] * buses[i].cos_phi for t in 1:T, i in Ns+1:N]
# base_daily_profiles
load_profiles
#maximum(sum(base_load_profiles, dims=2))
#size(sum(base_load_profiles, dims=2))

48×2 Matrix{Float64}:
 0.482429  0.101669
 0.319094  0.101669
 0.185928  0.101669
 0.265269  0.101669
 0.301518  0.101669
 0.307864  0.101669
 0.335939  0.101669
 0.488708  0.609216
 3.69171   0.601076
 2.50983   0.604365
 0.881196  0.529077
 0.635065  0.384581
 0.515056  0.152171
 ⋮         
 0.264073  0.291517
 0.744243  0.286168
 0.854085  0.205364
 0.853354  0.345076
 3.10701   0.812553
 3.2941    0.626393
 1.9148    1.2061
 2.4243    1.06842
 1.94078   1.54397
 1.71521   1.42831
 1.57168   0.995856
 0.264073  0.37046

In [19]:
using JuMP
# -- Print case description --
function print_case_description(model::JuMP.AbstractModel)
    # Fetch data
    
    
    DELTA_T         = model[:delta_t]
    NB_PROFILES     = model[:nb_sign_days]
    nb_time_steps   = model[:time_steps]  
    bilevel         = model[:bilevel]
    N               = get_nb_buses(network)
    Ns              = get_nb_substations(network)
    buses           = network.buses
    BASE_POWER      = network.pu_basis.base_power

    # 1. Energy consumed
    energy_consumed = 0
    for i in Ns+1:N
        P_cons = sum(buses[i].load_profile.time_serie * buses[i].cos_phi)
        energy_consumed += P_cons * BASE_POWER * DELTA_T / 60 * 365 / NB_PROFILES
    end
    
    # 2. P_sub max and min
    peak_sub_P = value.(sum(model[:P_sub], dims=2)) * BASE_POWER
    peak_sub_max = maximum(peak_sub_P) * BASE_POWER
    peak_sub_min = minimum(peak_sub_P) * BASE_POWER

    # Build table
    case_headers = (
        ["Model", "Objective", "Gap", "Solve Time", "Time periods", "Energy consumed", "Peak sub max", "Peak sub min"], ["", "kEUR/year", "%", "sec.","", "MWh/year", "MW on 1 DT", "MW on 1 DT"])
    case_params = sig_round([(bilevel ? "Bilevel" : "Single Level") objective_value(model) relative_gap(model)*100 solve_time(model) nb_time_steps energy_consumed peak_sub_max peak_sub_min])
    pretty_table(case_params, header=case_headers)

    # Returns the table as a Matrix
    return vcat([i[j] for i in case_headers, j in 1:length(case_headers[1])], case_params)
end

print_case_description (generic function with 1 method)

In [20]:
print_case_description(model)

2
┌─────────┬───────────┬────────┬────────────┬──────────────┬─────────────────┬──────────────┬──────────────┐
│   Model │ Objective │    Gap │ Solve Time │ Time periods │ Energy consumed │ Peak sub max │ Peak sub min │
│         │ kEUR/year │      % │       sec. │              │        MWh/year │   MW on 1 DT │   MW on 1 DT │
├─────────┼───────────┼────────┼────────────┼──────────────┼─────────────────┼──────────────┼──────────────┤
│ Bilevel │     46.34 │ 0.1602 │     0.1253 │            4 │         10680.0 │         2.19 │       0.5098 │
└─────────┴───────────┴────────┴────────────┴──────────────┴─────────────────┴──────────────┴──────────────┘


3×8 Matrix{Any}:
 "Model"      "Objective"   "Gap"  …   "Peak sub max"   "Peak sub min"
 ""           "kEUR/year"   "%"        "MW on 1 DT"     "MW on 1 DT"
 "Bilevel"  46.34          0.1602     2.19             0.5098

In [ ]:
Bilevel.print_network_results(model)

In [ ]:
using JuMP
function print_cost_results(model; latex=false)
    network = model[:network_data]
    DSO_costs = model[:DSO_costs]


    L = get_nb_lines(network)
    K = get_nb_conductors(network)
    lines = network.lines 
    conductors = network.conductors

    # Investements to build new lines in k€
    DSO_fixed_line = value(sum(model[:Alpha][l, k] * lines[l].length * conductors[k].cost for l in 1:L, k in 1:K))

    BASE_POWER = network.pu_basis.base_power
    SUB_COST = DSO_costs.substation
    Ns = get_nb_substations(network)
   
    # Investements to build new substations in k€
    DSO_fixed_sub = value(sum(model[:S_sub_capa][i] * BASE_POWER * SUB_COST for i in 1:Ns))

   
    # Operational costs of losses on the length of the horizon
    DSO_loss = value(model[:DSO_loss_costs]) * DSO_costs.amortization

    # DSO future value: DSO should get back what he must pay per year for the investment + a certain margin
    DSO_future_value = value(model[:DSO_fixed_costs]) * ((1 + DSO_costs.interest_rate)^DSO_costs.amortization) + value(model[:DSO_loss_costs]) * DSO_costs.amortization

    # DSO revenues
    DSO_revenues = value(sum(model[:grid_costs])) * DSO_costs.amortization
    # Other costs
    user_costs = value(sum(model[:user_costs]))
    PV_costs = value(sum(model[:PV_costs]))
    grid_costs = value(sum(model[:grid_costs]))
    energy_costs = value(sum(model[:energy_costs]))
    energy_revenues = value(sum(model[:energy_revenues]))
    if model[:storage]
        stor_costs = value(sum(model[:storage_costs]))
        kpis_header = (["DSO fixed line", "DSO fixed sub", "DSO loss", "DSO future value", "DSO revenues", "User", "PV", "Storage", "Grid connection", "Energy imported", "Energy exported"],
        ["kEUR", "kEUR", "kEUR", "kEUR", "kEUR", "kEUR/year", "kEUR/year", "kEUR/year", "kEUR/year", "kEUR/year", "kEUR/year"])
        kpis = sig_round([DSO_fixed_line DSO_fixed_sub DSO_loss DSO_future_value DSO_revenues user_costs PV_costs stor_costs grid_costs energy_costs energy_revenues])
    end
    kpis_header = (["DSO fixed line", "DSO fixed sub", "DSO loss", "DSO future value", "DSO revenues", "User", "PV", "Grid connection", "Energy imported", "Energy exported"],
        ["kEUR", "kEUR", "kEUR", "kEUR", "kEUR", "kEUR/year", "kEUR/year", "kEUR/year", "kEUR/year", "kEUR/year"])
    kpis = sig_round([DSO_fixed_line DSO_fixed_sub DSO_loss DSO_future_value DSO_revenues user_costs PV_costs grid_costs energy_costs energy_revenues])
    if latex
        pretty_table(kpis, header=kpis_header, backend=Val(:latex))
    else
        pretty_table(kpis, header=kpis_header)
    end
    # returns the table as a Matrix
    return vcat([i[j] for i in kpis_header, j in 1:length(kpis_header[1])], kpis)
end

In [ ]:
function sig_round(x)
    for (i, a) in enumerate(x)
        if isa(a, AbstractFloat)
            x[i] = round(a, sigdigits=4)
            if abs(a) < 1e-3
                x[i] = 0
            end
        end
    end
    return x
end

In [ ]:
print_cost_results(model)

In [ ]:
function print_case_results(model; latex=false)
    DAYS_A_YEAR = 365
    DELTA_T = model[:delta_t]
    NB_PROFILES = model[:nb_sign_days]
   
    MULTIPLIER = DELTA_T/60 * DAYS_A_YEAR / NB_PROFILES
    # PV penetration : p_pv max = peak PV 
    PV_penetration = sum(value.(model[:p_pv_max]))
    # PV energy
    PV_energy = sum(value.(model[:p_pv])) * MULTIPLIER / PV_penetration

    # PV potential
    network = model[:network_data]
    Nu = get_nb_loads(network)
    Ns = get_nb_substations(network)
    T  = model[:time_steps]
    buses = network.buses 
    PV_prod = [(isnothing(buses[Ns + i].PV_installation) ? zeros(Float64, T) : buses[Ns + i].PV_installation.profile.time_serie) for i in 1:Nu]

    PV_potential = sum(PV_prod[i][t] * value(model[:p_pv_max][i]) for i in 1:Nu, t in 1:T) * MULTIPLIER / PV_penetration

    # Energy exchanges with grid
    energy_to_grid = value(sum(model[:p_exp])) * MULTIPLIER
    energy_from_grid = value(sum(model[:p_imp])) * MULTIPLIER

    # Substations capa
    sub_init_capa = [buses[i].S_rating for i in 1:Ns]
    substation1_capacity = sub_init_capa[1]
    substation2_capacity = sub_init_capa[2]
    kpis_header = (["PV penetration", "PV production", "PV potential", "Energy bought from grid", "Energy sold to grid", "Substation1 capacity", "Substation2 capacity"], ["MVA", "MWh/MVA/year", "MWh/MVA/year", "MWh/year", "MWh/year", "MVA", "MVA"])
    kpis = sig_round([PV_penetration PV_energy PV_potential energy_from_grid energy_to_grid substation1_capacity substation2_capacity])
    if latex
        pretty_table(kpis, header=kpis_header, backend=Val(:latex))
    else
        pretty_table(kpis, header=kpis_header)
    end
    # returns the table as a Matrix
    return vcat([i[j] for i in kpis_header, j in 1:length(kpis_header[1])], kpis)
end

In [ ]:
print_case_results(model)

In [ ]:
function print_network_results(model; latex=false)
    network = model[:network_data]
    nb_lines_built = 0
    for l in network.lines 
        if l.built 
            nb_lines_built += 1
        end
    end

    Ns = get_nb_substations(network)
    buses = network.buses
    nb_substations = 0
    for i in 1:Ns
        if buses[i].built
            nb_substations += 1
        end
    end
 
    K = get_nb_conductors(network)
    cond_string = ["Nb lines cond "*string(i) for i in 1:K]

    lines_cond = [[i for (i, a) in enumerate(value.(model[:Alpha])[:, j]) if isapprox(a, 1, rtol=1e-2)] for j in 1:K]

    kpis_header = ([["Nb lines built", "Nb substations"];cond_string], [["-", "-"];fill("-", 1:K)])
    kpis = reshape(string.([[nb_lines_built, nb_substations];lines_cond]),1,:)
    if latex
        pretty_table(kpis, header=kpis_header, backend=Val(:latex))
    else
        pretty_table(kpis, header=kpis_header)
    end
    # returns the table as a Matrix
    return vcat([i[j] for i in kpis_header, j in 1:length(kpis_header[1])], kpis)
end


In [ ]:
print_network_results(model)

In [ ]:
using Statistics
function print_pv_results(model; latex=true)
    DAYS_A_YEAR = 365
    DELTA_T = model[:delta_t]
    NB_PROFILES = model[:nb_sign_days]
    MULTIPLIER = DELTA_T/60 * DAYS_A_YEAR / NB_PROFILES

    # PV penetration
    PV_penetration = sum(value.(model[:p_pv_max]))

    # PV energy
    PV_energy = sum(value.(model[:p_pv])) * MULTIPLIER / PV_penetration # percentage

    # PV potential
    network = model[:network_data]
    Nu = get_nb_loads(network)
    Ns = get_nb_substations(network)
    T  = model[:time_steps]
    buses = network.buses 
    PV_prod = [(isnothing(buses[Ns + i].PV_installation) ? 0.0 : buses[Ns + i].PV_installation.profile.time_serie[t]) for t in 1:T, i in 1:Nu]

    PV_potential = sum(PV_prod[t, i] * value(model[:p_pv_max][i]) for t in 1:T, i in 1:Nu) * MULTIPLIER / PV_penetration

    # Self-sufficiency
    P_consumed = [buses[Ns + i].load_profile.time_serie[t] * buses[Ns + i].cos_phi for t in 1:T, i in 1:Nu]

   

    # potential_array = PV_prod' .* value.(model[:p_pv_max])
    # index_non_zeros = potential_array .!= 0

    # # self_sufficiency = mean((value.(model[:p_pv]) .- value.(model[:p_exp])) ./ P_consumed)

    # # self_consumption = mean((value.(model[:p_pv])[index_non_zeros] - value.(model[:p_exp])[index_non_zeros]) ./ value.(model[:p_pv])[index_non_zeros])

    # # production_ratio = mean(value.(model[:p_pv])[index_non_zeros] ./
    # # potential_array[index_non_zeros])

    # Self-suffiency and self-consumption ? => ASK BERTRAND
    avg_self_sufficiency = 0
    avg_self_consumption = 0
    count = 0
    for t in 1:T
        # self sufficiency = total PV prod at time t - total PV_export at time t)/total cons at time t
        avg_self_sufficiency += (value(sum(model[:p_pv][t, :])) - value(sum(model[:p_exp][t, :])))/(sum(P_consumed[t, :])/T)


        index_nonzeros = PV_prod[t, :] != 0
        avg_self_consumption += (sum(value.(model[:p_pv][t, :])[index_nonzeros]) - sum(value(model[:p_exp][t, :])[index_nonzeros]))/(sum(PV_prod[t, :][index_nonzeros])/length(index_nonzeros))

        production_ratio += (sum(value.(model[:p_pv][t, :])[index_nonzeros]) - sum(value.(model[:p_exp][t, :])[index_nonzeros]))/(PV_potential[t,:][index_nonzeros]/length(index_nonzeros))
    end
    
    # PV_costs 
    User_costs = model[:User_costs]
    pv_cost = value(sum(model[:s_conv_pv])) * User_costs.PV_conv/ User_costs.amortization_PVC + value(sum(model[:p_pv_max])) * User_costs.PV / User_costs.amortization_PV

    # Pv prod
    pv_production = sum(value.(model[:p_pv])) #* MULTIPLIER
    LCOE_pv = pv_cost / pv_production
    kpis_header = (["PV penetration", "PV production", "PV potential", "Avg self sufficiency", "Avg self consumption", "production ratio", "LCOE pv"],
        ["MVA", "MWh/MVA/year", "MWh/MVA/year", "average, [-]", "average, [-]", "average, [-]", "kEUR/MWh"])
    kpis = sig_round([PV_penetration PV_energy PV_potential avg_self_sufficiency self_consumption production_ratio LCOE_pv])
    if latex
        pretty_table(kpis, header=kpis_header, backend=Val(:latex))
    else
        pretty_table(kpis, header=kpis_header)
    end
    # returns the table as a Matrix
    return vcat([i[j] for i in kpis_header, j in 1:length(kpis_header[1])], kpis)
end

In [ ]:
print_pv_results(model)

In [ ]:
# in progress
function print_grid_results(model; latex=true)
    network = model[:network_data]
    DELTA_T = model[:delta_t]
    NB_PROFILES = model[:nb_sign_days]
    Nu = get_nb_loads(network)
    Ns = get_nb_substations(network)
    DAYS_A_YEAR = 365
    MULTIPLIER = DELTA_T/60 * DAYS_A_YEAR / NB_PROFILES
    BASE_POWER = network.pu_basis.base_power

    # Average grid capacity
    grid_capacity = (value(sum(model[:s_grid_max])) * BASE_POWER) / Nu

    # Energy exchanges
    energy_from_grid = value(sum(model[:p_imp])) * MULTIPLIER * BASE_POWER
    energy_to_grid = value(sum(model[:p_exp])) * MULTIPLIER * BASE_POWER

    # Substations
    substation_string = ["Substation "*string(i)*" capacity" for i in 1:Ns]
    buses = network.buses
    substation_capacities = [buses[i].S_rating*BASE_POWER for i in 1:Ns]

    # Computation of the losses
    lines = network.lines
    conductors = network.conductors
    T = model[:time_steps]
    L = get_nb_lines(network)
    K = get_nb_conductors(network)

    loss = sum(lines[l].length * conductors[k].r *
    model[:I_sqr_k][t, l, k] for t in 1:T, l in 1:L, k in 1:K) * MULTIPLIER * BASE_POWER

    # Total grid costs
    User_costs = model[:User_costs]
    grid_cost = (value(sum(model[:p_imp])) * User_costs.EI * MULTIPLIER + value(sum(model[:p_imp])) * User_costs.DSOEI * MULTIPLIER + value(sum(model[:s_grid_max])) * User_costs.GCC) * BASE_POWER

    # Total production
    grid_production = value(sum(model[:p_imp])) * MULTIPLIER
    LCOE_grid = grid_cost / grid_production
    kpis_header = ([["Grid capacity", "Energy bought from grid", "Energy sold to grid"];substation_string; ["loss", "LCOE grid"]],
        [["average, MVA", "MWh/year", "MWh/year"];fill("MVA",1:Ns);["MWh/year", "kEUR/MWh"]])
    kpis = reshape(sig_round([[grid_capacity, energy_from_grid, energy_to_grid];substation_capacities;[loss, LCOE_grid]]),1,:)
    if latex
        pretty_table(kpis, header=kpis_header, backend=Val(:latex))
    else
        pretty_table(kpis, header=kpis_header)
    end
    # returns the table as a Matrix
    return vcat([i[j] for i in kpis_header, j in 1:length(kpis_header[1])], kpis)
end

In [ ]:
using JuMP
print_grid_results(model)

In [ ]:
function print_storage_results(model; latex=true)
    network = model[:network_data]
    Nu = get_nb_loads(model)

    # storage capacity
    storage_capacity = value(sum(model[:storage_capacity])) / Nu

    # Avg storage state
    avg_storage_state = mean(value.(model[:storage_state]))
    kpis_header = (["Storage capacity", "Storage state"],
        ["average, MWh", "average, [-]"])
    kpis = sig_round([storage_capacity avg_storage_state])
    if latex
        pretty_table(kpis, header=kpis_header, backend=Val(:latex))
    else
        pretty_table(kpis, header=kpis_header)
    end
    # returns the table as a Matrix
    return vcat([i[j] for i in kpis_header, j in 1:length(kpis_header[1])], kpis)
end

In [ ]:
function print_summary(model; latex=true)
    DSO_costs = model[:DSO_costs]
    T = model[:time_steps]
    network = model[:network_data]
    Nu = get_nb_loads(network)
    Ns = get_nb_substations(network)

    DSO_capex = value(model[:DSO_fixed_costs]) / DSO_costs.amortization / 1000
    DSO_opex = value(model[:DSO_loss_costs]) / 1000
    PV_costs = value(sum(model[:PV_costs])) / 1000
    grid_costs = value(sum(model[:grid_costs])) / 1000
    net_energy_costs = (value(sum(model[:energy_costs])) - value(sum(model[:energy_revenues]))) / 1000

    buses = network.buses
    P_consumed = [buses[Ns + i].load_profile.time_serie[t] * buses[Ns + i].cos_phi for t in 1:T, i in 1:Nu]

    PV_prod = [(isnothing(buses[Ns + i].PV_installation) ? 0.0 : buses[Ns + i].PV_installation.profile.time_serie[t]) for t in 1:T, i in 1:Nu]


    #self_sufficiency = mean(min.(value.(model[:p_pv]), P_consumed) ./ P_consumed) * 100
    #potential_array = (PV_prod .* value.(model[:p_pv_max]))
    #index_non_zeros = potential_array .!= 0
    #self_consumption = mean(min.(potential_array, P_consumed)[index_non_zeros] ./
    #                        potential_array[index_non_zeros]) * 100
    kpis_header = (["CAPEX", "OPEX", "UPVC", "UGCC", "UNEEC", "USS", "USC"],
        ["M€/y", "M€/y", "M€/y", "M€/y", "M€/y", "%", "%"])
    kpis = sig_round([DSO_capex DSO_opex PV_costs grid_costs net_energy_costs self_sufficiency self_consumption])
    if latex
        pretty_table(kpis, header=kpis_header, backend=Val(:latex))
    else
        pretty_table(kpis, header=kpis_header)
    end
    # returns the table as a Matrix
    return vcat([i[j] for i in kpis_header, j in 1:length(kpis_header[1])], kpis)
end

In [ ]:
print_summary(model)